<h1>Case Study: Flowers CNN Classifier</h1>
Flowers dataset (https://www.kaggle.com/alxmamaev/flowers-recognition) 

*Make sure you activate Hardware accelerator in the settings. 
Go to Runtime --> Change runtime type --> Set Hardware Accelerator to GPU*

In [ ]:
%tensorflow_version 2.x

**Mounting Google Drive on to the Google Colab instance**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New section

**Set the appropriate path for the datsaet zip provided**

In [10]:
images_path = "/content/drive/MyDrive/PM Document Repo/Currently Used Decks/Mentor Session Material/Module 9 - Computer Vision/Week 1 - CV  - Mentor deck/Copy of flowers.zip"

**Extracting the dataset.zip to the present working directory**

In [11]:
from zipfile import ZipFile
with ZipFile(images_path,'r') as zip:
  zip.extractall()

*Check the list of files in the pwd(present working directory) by running command 'ls' and ensure 'dataset' folder has been generated*

In [12]:
!ls

drive  flowers	sample_data


**Importing required Keras modules**

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

<h2>Building the CNN Classifier</h2>

In [16]:
# Initialising the CNN classifier
classifier = Sequential()

# Add a Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu', padding = 'same'))

# Add a Max Pooling layer of size 2X2
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Add another Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))

# Adding another pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Add another Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))

# Adding another pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening the layer before fully connected layers
classifier.add(Flatten())

# Adding a fully connected layer with 512 neurons
classifier.add(Dense(units = 512, activation = 'relu'))

# Adding dropout with probability 0.5
classifier.add(Dropout(0.5))


# Adding a fully connected layer with 128 neurons
classifier.add(Dense(units = 128, activation = 'relu'))


# The final output layer with 5 neuron to predict the categorical classifcation
classifier.add(Dense(units = 5, activation = 'softmax'))

**Compiling the CNN classifier with Adam optimizer (default Learning rate and other parameters)
and Categorical Crossentropy as loss function and Accuracy as the metric to monitor**

*Optionally you can use an optimizer with custom learning rate and passing it to the optimizer parameter of compile*

*Eg: keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)*

In [18]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

**Dataset Pre-processing**

*ImageDataGenerator is a powerful preprocessing utility to generate training and 
testing data with common data augmentation techniques. It can also be used to  
generate training data from Images stored in hierarchical directory structures
For more options of ImageDataGenerator go to https://keras.io/preprocessing/image/*

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Create data generator for training data with data augmentation and normalizing all
# values by 255
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1./255)

# Setting training data generator's source directory
# Setting the target size to resize all the images to (64,64) as the model input layer expects 32X32 images

training_set = train_datagen.flow_from_directory('./flowers/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

# Setting testing data generator's source directory
test_set = test_datagen.flow_from_directory('./flowers/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')


# There are 3823 training images and 500 test images in total
classifier.fit_generator(training_set,
                         steps_per_epoch = int(3823/32),
                         epochs = 20,
                         validation_data = test_set,
                         validation_steps = int(500/32))


Found 3823 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
119/119 [==============================] - 23s 138ms/step - loss: 1.4854 - accuracy: 0.3380 - val_loss: 1.1589 - val_accuracy: 0.4875
Epoch 2/20
119/119 [==============================] - 16s 136ms/step - loss: 1.1459 - accuracy: 0.5275 - val_loss: 1.1126 - val_accuracy: 0.5292
Epoch 3/20
119/119 [==============================] - 16s 135ms/step - loss: 1.0254 - accuracy: 0.6039 - val_loss: 1.0891 - val_accuracy: 0.5292
Epoch 4/20
119/119 [==============================] - 16s 135ms/step - loss: 0.9488 - accuracy: 0.6331 - val_loss: 1.1369 - val_accuracy: 0.5437
Epoch 5/20
119/119 [==============================] - 16s 134ms/step - loss: 0.8868 - accuracy: 0.6696 - val_loss: 0.9763 - val_accuracy: 0.5708
Epoch 6/20
119/119 [==============================] - 16s 135ms/step - loss: 0.8695 - accuracy: 0.6748 - val_loss: 0.9264 - val_accuracy: 0.6125
Epoch 7/20
119/119 [==============================] - 16s 133ms/step - loss: 0.7892 - accuracy: 0.6946 - val_loss: 0.9599 - val_ac

**Always save the model and its weights after training**

In [20]:
classifier.save('./classifier.h5')

classifier.save_weights('./classifier_weights.h5')

*Check the current directory if the weights have been saved*

In [21]:
!ls

classifier.h5  classifier_weights.h5  drive  flowers  sample_data


<h2>Let's test the model now</h2>

**Load the pre-trained saved model**

In [23]:

from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the pre trained model from the HDF5 file saved previously
pretrained_model = load_model('./classifier.h5')
pretrained_model.load_weights('./classifier_weights.h5')

**Testing the model on a test image from one of the test folders**

In [24]:
import cv2
test_image = cv2.imread('./flowers/test/daisy/100080576_f52e8ee070_n.jpg')
# Resize the image to 64X64 shape to be compatible with the model
test_image = cv2.resize(test_image,(64,64))

# Check if the size of the Image array is compatible with Keras model
print(test_image.shape)

# If not compatible expand the dimensions to match with the Keras Input
test_image = np.expand_dims(test_image, axis = 0)
test_image =test_image*1/255.0

#Check the size of the Image array again
print('After expand_dims: '+ str(test_image.shape))


#Predict the result of the test image
result = classifier.predict(test_image)

# Check the indices Image Data Generator has allotted to each folder
classes_dict = training_set.class_indices
print(classes_dict)

# Creating a list of classes in test set for showing the result as the folder name
prediction_class = []
for class_name,index in classes_dict.items():
  prediction_class.append(class_name)
  
print(result[0])

# Index of the class with maximum probability
predicted_index = np.argmax(result[0])

# Print the name of the class
print(prediction_class[predicted_index])

(64, 64, 3)
After expand_dims: (1, 64, 64, 3)
{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}
[0.19680828 0.71740514 0.05897095 0.0025997  0.02421595]
dandelion


**Generating a report on the test data**

In [25]:
# Re-initalizing the test data generator with shuffle=False to create the confusion matrix
test_set = test_datagen.flow_from_directory('./flowers/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode = 'categorical')

# Predict the whole generator to get predictions
Y_pred = classifier.predict_generator(test_set, int(500/32+1))

# Find out the predictions classes with maximum probability
y_pred = np.argmax(Y_pred, axis=1)

# Utilities for confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

# Printing the confusion matrix based on the actual data vs predicted data. 
print(confusion_matrix(test_set.classes, y_pred))

# Printing the classification report
print(classification_report(test_set.classes, y_pred, target_names=prediction_class))

Found 500 images belonging to 5 classes.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


[[66 16 11  4  3]
 [11 71  1 17  0]
 [10  6 64  2 18]
 [ 4  4  2 89  1]
 [ 4  4 46  7 39]]
              precision    recall  f1-score   support

       daisy       0.69      0.66      0.68       100
   dandelion       0.70      0.71      0.71       100
        rose       0.52      0.64      0.57       100
   sunflower       0.75      0.89      0.81       100
       tulip       0.64      0.39      0.48       100

    accuracy                           0.66       500
   macro avg       0.66      0.66      0.65       500
weighted avg       0.66      0.66      0.65       500



**We can play around with the model by adding regularization to the layers, adding more convolutional layers, etc., to improve the testing accuracy**